In [1]:
%use dataframe, kandy

In [2]:
USE {
    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [3]:
import java.nio.file.Path
import kotlin.io.path.*

fun String.parse(s1: String, s2: String): Pair<Int, Int> {
    return Regex("""$s1 = (\d+), $s2 = (\d+)""").find(this)!!.groupValues.let {
        it[1].toInt() to it[2].toInt()
    }
}

fun String.parseTime(s1: String, s2: String): Pair<Long, Long> {
    return Regex("""$s1 = (\d+) ms, $s2 = (\d+) ms""").find(this)!!.groupValues.let {
        it[1].toLong() to it[2].toLong()
    }
}

fun mkDataFrameFromLog(path: Path): DataFrame<*> {
    val benchmark = mutableListOf<String>()
    val cnt = mutableListOf<Int>()
    val sat = mutableListOf<Int>()
    val unsat = mutableListOf<Int>()
    val unknown = mutableListOf<Int>()
    val reusedUnsat = mutableListOf<Int>()
    val solvingTime = mutableListOf<Long>()
    val checkingTime = mutableListOf<Long>()
    val updatingTime = mutableListOf<Long>()

    path.readLines()
        .chunked(8)
        .dropLastWhile { it.size != 8 }
        .forEach {
            val benchmarkR = it[1].substringBefore(" ")
            val (cntR, satR) = it[2].parse("cnt", "sat")
            val (unsatR, unknownR) = it[3].parse("unsat", "unknown")
            val reusedUnsatR = Regex("reusedUnsat = (\\d+)").find(it[4])!!.groupValues[1].toInt()
            val (solvingTimeR, checkingTimeR) = it[5].parseTime("solvingTime", "checkingTime")
            val (updatingTimeR, _) = it[6].parseTime("updatingTime", "totalTime")

            benchmark += benchmarkR
            cnt += cntR
            sat += satR
            unsat += unsatR
            unknown += unknownR
            reusedUnsat += reusedUnsatR
            solvingTime += solvingTimeR
            checkingTime += checkingTimeR
            updatingTime += updatingTimeR
        }
    return dataFrameOf(
        "benchmark" to benchmark,
        "cnt" to cnt,
        "sat" to sat,
        "unsat" to unsat,
        "unknown" to unknown,
        "reusedUnsat" to reusedUnsat,
        "solvingTime" to solvingTime,
        "checkingTime" to checkingTime,
        "updatingTime" to updatingTime,
    )
}

In [10]:
fun DataFrame<*>.addConfigColumn(name: String): DataFrame<*> =
    add(column(List(rowsCount()) { name }).rename("config"))

fun DataFrame<*>.addProjectColumn(): DataFrame<*> =
    add(this["benchmark"].map { (it as String).substringBefore('-') }.rename("project"))

fun DataFrame<*>.timeToLong(): DataFrame<*> = columns().filter { it.name.endsWith("Time") }.toTypedArray().let { cols ->
    remove(*cols).addAll(cols.map { it.convertToLong() })
}

val folder = "../output/cache"

val data = Path(folder).listDirectoryEntries().map {
    if (it.div("smtData.csv").exists()) {
        DataFrame.readCSV((it / "smtData.csv").toFile()).timeToLong()
    } else {
        mkDataFrameFromLog(it / "cache-a-lot.log")
    }.addConfigColumn(it.name).addProjectColumn().sortBy("project")
}.concat()

data.columnNames()

[benchmark, cnt, sat, unsat, unknown, reusedUnsat, solvingTime, checkingTime, updatingTime, config, project]

In [11]:
val configs = data.config.countDistinct()
val fullBenchs = data.groupBy("benchmark")
    .aggregate { it.config.countDistinct() into "configs" }
    .filter { it["configs"] == configs }
    .benchmark.values.toList()
val fullData = data.filter { benchmark in fullBenchs }

"Excluded: ${data.benchmark.countDistinct() - fullBenchs.size}"

Excluded: 0

In [61]:
import org.jetbrains.kotlinx.kandy.ir.Plot

fun plotByConfig(
    chunkSize: Int,
    width: Int,
    height: Int,
    filter: (String) -> Boolean = { true },
    sortBy: (String) -> String = { it },
    forceSize: Boolean = true,
    block: (String) -> Plot
) = plotBunch {
    fullData.config.distinct()
        .values
        .sortedBy { sortBy(it) }
        .filter { filter(it) }
        .chunked(chunkSize)
        .forEachIndexed { i, confs ->
            confs.forEachIndexed { j, conf ->
                if (forceSize) {
                    add(block(conf), width * j, height * i, width, height)
                } else {
                    add(block(conf), width * j, height * i)
                }
            }
        }
}

In [83]:
val interestedFilter: (String) -> Boolean = { "random" !in it && it.endsWith("tt") }
val interestedSort: (String) -> String = { it.reversed() }

In [137]:
plotByConfig(2, 600, 400, { it != "empty-tt" && interestedFilter(it) }, interestedSort) { conf ->
    fullData
        .filter { config == conf }
        .groupBy("project")
        .aggregate {
            val total = unsat.sum()
            val reused = reusedUnsat.sum()
            (total - reused).toDouble() / 1 into "unique unsat"
            reused.toDouble() / 1 into "reused unsat"
        }
        .gather("reused unsat", "unique unsat")
        .into("name", "value")
        .sortBy("project")
        .groupBy("name")
        .plot {
            bars {
                x(project)
                y("value")
                fillColor("name") {
                    scale = categorical(
                        "unique unsat" to Color.GREY,
                        "reused unsat" to Color.ORANGE,
                    )
                }
                position = Position.stack()
            }
            layout {
                title = conf
            }
        }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="HlOhlw"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"kind":"ggbunch",
"items":[{
"x":0,
"width":600,
"y":0,
"feature_spec":{
"ggtitle":{
"text":"simple-full-tt"
},
"mapping":{
},
"data":{
"&merged_groups":["reused unsat","reused unsat","reused unsat","reused unsat","unique unsat","unique unsat","unique unsat","unique unsat"],
"name":["reused unsat","reused unsat","reused unsat","reused unsat","unique unsat","unique unsat","unique unsat","unique unsat"],
"project":["FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON"],
"value":[299.0,247.0,11.0,985.0,601.0,868.0,75.0,258.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#a39999","#fc8452"],
"limits":["unique unsat","reused unsat"]
}],
"layers":[{
"mapping":{
"x":"project",
"y":"value",
"fill":"name",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}]
},
"height":400
},{
"x":600,
"width":600,
"y":0,
"feature_spec":{
"ggtitle":{
"text":"fullopt-full-tt"
},
"mapping":{
},
"data":{
"&merged_groups":["reused unsat","reused unsat","reused unsat","reused unsat","unique unsat","unique unsat","unique unsat","unique unsat"],
"name":["reused unsat","reused unsat","reused unsat","reused unsat","unique unsat","unique unsat","unique unsat","unique unsat"],
"project":["FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON"],
"value":[420.0,504.0,62.0,1112.0,480.0,615.0,24.0,133.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#a39999","#fc8452"],
"limits":["unique unsat","reused unsat"]
}],
"layers":[{
"mapping":{
"x":"project",
"y":"value",
"fill":"name",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}]
},
"height":400
}]
};
 var plotContainer = document.getElementById("HlOhlw");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 FASTJSON 
 
 
 
 
 
 
 
 
 GUAVA 
 
 
 
 
 
 
 
 
 SEATA 
 
 
 
 
 
 
 
 
 SPOON 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 simple-full-tt 
 
 
 
 
 value 
 
 
 
 
 project 
 
 
 
 
 
 
 
 
 name 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 unique unsat 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 reused unsat 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 FASTJSON 
 
 
 
 
 
 
 
 
 GUAVA 
 
 
 
 
 
 
 
 
 SEATA 
 
 
 
 
 
 
 
 
 SPOON 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 fullopt-full-tt 
 
 
 
 
 value 
 
 
 
 
 project 
 
 
 
 
 
 
 
 
 name 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 unique unsat 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 reused unsat

In [85]:
plotByConfig(3, 350, 400, interestedFilter, interestedSort, false) { conf ->
    fullData
        .filter { config == conf }
        .groupBy("project")
        .aggregate {
            (solvingTime.sum().toDouble() / cnt.sum()) into "solvingTime"
            (checkingTime.sum().toDouble() / cnt.sum()) into "checkingTime"
            (updatingTime.sum().toDouble() / cnt.sum()) into "updatingTime"
        }
        .gather(
            "solvingTime",
            "checkingTime",
            "updatingTime",
        )
        .into("type", "value")
        .sortBy("project")
        .groupBy("type")
        .plot {
            y {
                limits = 0..600
            }
            bars {
                x(project)
                y("value") {
                    axis.name = "avg. time (ms)"
                }
                fillColor("type") {
                    scale = categorical(
                        "solvingTime" to Color.ORANGE,
                        "checkingTime" to Color.GREY,
                        "updatingTime" to Color.BLUE,
                    )
                    if (conf != "empty-tt") {
                        legend {
                            type = LegendType.None
                        }
                    }
                }
                position = Position.stack()
            }
            layout { 
                title = conf
                if (conf == "empty-tt") {
                    size = 500 to 400
                } else {
                    size = 350 to 400
                }
            }
        }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="1m5mND"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"kind":"ggbunch",
"items":[{
"x":0,
"width":null,
"y":0,
"feature_spec":{
"ggtitle":{
"text":"simple-full-tt"
},
"mapping":{
},
"data":{
"&merged_groups":["solvingTime","solvingTime","solvingTime","solvingTime","checkingTime","checkingTime","checkingTime","checkingTime","updatingTime","updatingTime","updatingTime","updatingTime"],
"project":["FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON"],
"type":["solvingTime","solvingTime","solvingTime","solvingTime","checkingTime","checkingTime","checkingTime","checkingTime","updatingTime","updatingTime","updatingTime","updatingTime"],
"value":[350.8719927371766,515.3255907088507,69.08552631578948,344.2914349276974,0.6836132546527462,56.39767721265519,0.0,0.41212458286985537,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":350.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"name":"avg. time (ms)",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#fc8452","#a39999","#5470c6"],
"limits":["solvingTime","checkingTime","updatingTime"],
"guide":"none"
},{
"aesthetic":"y",
"limits":[0.0,600.0]
}],
"layers":[{
"mapping":{
"x":"project",
"y":"value",
"fill":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}]
},
"height":null
},{
"x":350,
"width":null,
"y":0,
"feature_spec":{
"ggtitle":{
"text":"fullopt-full-tt"
},
"mapping":{
},
"data":{
"&merged_groups":["solvingTime","solvingTime","solvingTime","solvingTime","checkingTime","checkingTime","checkingTime","checkingTime","updatingTime","updatingTime","updatingTime","updatingTime"],
"project":["FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON"],
"type":["solvingTime","solvingTime","solvingTime","solvingTime","checkingTime","checkingTime","checkingTime","checkingTime","updatingTime","updatingTime","updatingTime","updatingTime"],
"value":[355.2482977757603,498.373247897477,65.65789473684211,330.63793103448273,36.558329550612804,46.0905086103324,4.703947368421052,51.47942157953281,0.0,0.0,0.0,5.561735261401557E-4]
},
"ggsize":{
"width":350.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"name":"avg. time (ms)",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#fc8452","#a39999","#5470c6"],
"limits":["solvingTime","checkingTime","updatingTime"],
"guide":"none"
},{
"aesthetic":"y",
"limits":[0.0,600.0]
}],
"layers":[{
"mapping":{
"x":"project",
"y":"value",
"fill":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}]
},
"height":null
},{
"x":700,
"width":null,
"y":0,
"feature_spec":{
"ggtitle":{
"text":"empty-tt"
},
"mapping":{
},
"data":{
"&merged_groups":["solvingTime","solvingTime","solvingTime","solvingTime","checkingTime","checkingTime","checkingTime","checkingTime","updatingTime","updatingTime","updatingTime","updatingTime"],
"project":["FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON"],
"type":["solvingTime","solvingTime","solvingTime","solvingTime","checkingTime","checkingTime","checkingTime","checkingTime","updatingTime","updatingTime","updatingTime","updatingTime"],
"value":[357.0531093962778,503.9094913896676,68.09868421052632,419.853170189099,0.0,0.0,0.0,0.0,0.0,4.0048057669203043E-4,0.0,0.0]
},
"ggsize":{
"width":500.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"name":"avg. time (ms)",
"limits":[null,nul

In [89]:
plotByConfig(3, 400, 400, interestedFilter, interestedSort) { conf ->
    fullData
        .filter { config == conf }
        .sortBy("project")
        .plot {
            y {
                limits = 0..2300
            }
            boxplot(
                project,
                column<Double>(
                    solvingTime.values.zip(cnt.values).map { it.first.toDouble() / it.second }
                ).named("avg. solving time (ms)")
            )
            layout { 
                title = conf
            }
        }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="yzQplR"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"kind":"ggbunch",
"items":[{
"x":0,
"width":400,
"y":0,
"feature_spec":{
"ggtitle":{
"text":"simple-full-tt"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"avg. solving time (ms)",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[0.0,2300.0]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"min":[0.09734513274336283,57.822546972860124,2.0,104.41666666666667],
"middle":[101.20481927710844,297.9798550724638,54.23809523809524,887.3695652173913],
"max":[1376.9776119402984,949.3905325443787,269.1666666666667,2216.8],
"lower":[2.793919578372048,147.16298098186644,5.75,219.55555555555554],
"upper":[555.8694036590716,591.6229646266142,187.2638888888889,1480.7391304347825],
"x":["FASTJSON","GUAVA","SEATA","SPOON"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["FASTJSON","GUAVA"],
"y":[1769.6470588235295,1382.4078091106292]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
},
"height":400
},{
"x":400,
"width":400,
"y":0,
"feature_spec":{
"ggtitle":{
"text":"fullopt-full-tt"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"avg. solving time (ms)",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[0.0,2300.0]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"min":[0.10619469026548672,57.07098121085595,2.0,105.16666666666667],
"middle":[102.09469126506025,297.8791304347826,54.714285714285715,917.4642857142857],
"max":[1280.0,892.8579881656805,284.6111111111111,2217.2],
"upper":[555.4388886021727,625.1077525311986,187.97222222222223,1405.391304347826],
"lower":[2.3549311926605503,141.55824579831932,3.75,213.7709923664122],
"x":["FASTJSON","GUAVA","SEATA","SPOON"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["FASTJSON","FASTJSON","GUAVA"],
"y":[1390.5820895522388,1768.1176470588234,1377.9045553145336]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
},
"height":400
},{
"x":800,
"width":400,
"y":0,
"feature_spec":{
"ggtitle":{
"text":"empty-tt"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"avg. solving time (ms)",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[0.0,2300.0]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"middle":[79.32528112449799,313.1020289855072,45.476190476190474,938.8913043478261],
"min":[1.2,58.791231732776616,2.0,103.75],
"max":[1307.975,967.2307692307693,303.5,2214.4],
"upper":[542.7304789306634,489.9376535095306,194.5,1575.9130434782608],
"lower":[4.59096417990843,149.51493707530858,6.6118421052631575,218.61111111111111],
"x":["FASTJSON","GUAVA","SEATA","SPOON"]
},
"sampling":"none",
"position":"ident

In [134]:
val tts = fullData.groupBy("project", "config")
    .aggregate { 
        solvingTime.sum().toDouble() / cnt.sum() into "solvingTime"
        checkingTime.sum().toDouble() / cnt.sum() into "checkingTime"
    }.sortBy("project", "config")
    .filter { 
        config.endsWith("tt") 
                && "random" !in config
    }
    .run {
        add(column(solvingTime.cast<Double>().values.zip(checkingTime.cast<Double>().values).map { it.first + it.second }).named("totalTime"))
    }
tts

project,config,solvingTime,checkingTime,totalTime
FASTJSON,empty-tt,357.053109,0.000000,357.053109
FASTJSON,fullopt-full-tt,355.248298,36.558330,391.806627
FASTJSON,simple-full-tt,350.871993,0.683613,351.555606
GUAVA,empty-tt,503.909491,0.000000,503.909491
GUAVA,fullopt-full-tt,498.373248,46.090509,544.463757
GUAVA,simple-full-tt,515.325591,56.397677,571.723268
SEATA,empty-tt,68.098684,0.000000,68.098684
SEATA,fullopt-full-tt,65.657895,4.703947,70.361842
SEATA,simple-full-tt,69.085526,0.000000,69.085526
SPOON,empty-tt,419.853170,0.000000,419.853170


In [135]:
val diffTts = tts.groupBy("project").updateGroups {
    val base = filter { config == "empty-tt" }.single().solvingTime
    groupBy("project", "config").aggregate { 100.0 * (base - solvingTime.single()) / base into "solvingTime (%)" }
}.concat()

diffTts

project,config,solvingTime (%)
FASTJSON,empty-tt,0.000000
FASTJSON,fullopt-full-tt,0.505474
FASTJSON,simple-full-tt,1.731148
GUAVA,empty-tt,0.000000
GUAVA,fullopt-full-tt,1.098658
GUAVA,simple-full-tt,-2.265506
SEATA,empty-tt,0.000000
SEATA,fullopt-full-tt,3.584195
SEATA,simple-full-tt,-1.449135
SPOON,empty-tt,0.000000


In [136]:
diffTts.filter { config != "empty-tt" }.groupBy("config").sortBy("config").plot { 
    bars { 
        x(project)
        y(`solvingTime (%)`)
        fillColor(config)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="EfAR4i"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"&merged_groups":["fullopt-full-tt","fullopt-full-tt","fullopt-full-tt","fullopt-full-tt","simple-full-tt","simple-full-tt","simple-full-tt","simple-full-tt"],
"project":["FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON"],
"config":["fullopt-full-tt","fullopt-full-tt","fullopt-full-tt","fullopt-full-tt","simple-full-tt","simple-full-tt","simple-full-tt","simple-full-tt"],
"solvingTime (%)":[0.5054742762411935,1.098658308047133,3.5841947637909306,21.249152201097903,1.731147691040293,-2.2655059121232357,-1.449135349241629,17.99718106865052]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"project",
"y":"solvingTime (%)",
"fill":"config",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("EfAR4i");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 FASTJSON 
 
 
 
 
 
 
 
 
 GUAVA 
 
 
 
 
 
 
 
 
 SEATA 
 
 
 
 
 
 
 
 
 SPOON 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 solvingTime (%) 
 
 
 
 
 project 
 
 
 
 
 
 
 
 
 config 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 fullopt-full-tt 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 simple-full-tt

In [147]:
fun res(name: String, unsat: Double, succ: Double, total: Double) {
    println(name)
    println("unsat: ${String.format("%.2f", 100.0 * (305746.0 - unsat) / 305746)}%")
    println("success: ${String.format("%.2f", 100.0 * (901134.0 - succ) / 901134)}%")
    println("total: ${String.format("%.2f", 100.0 * (2810097.0 - total) / 2810097)}%")
}

In [148]:
res("simple", 146564.0, 752142.0, 2689276.0)

simple
unsat: 52.06%
success: 16.53%
total: 4.30%


In [149]:
res("fullopt", 107510.0, 722478.0, 2631517.0)

fullopt
unsat: 64.84%
success: 19.83%
total: 6.35%
